### 1 模型构造

首先，回顾一下多层感知机

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0706,  0.1237, -0.0550,  0.0357,  0.0059,  0.1837, -0.0437,  0.0196,
         -0.0760, -0.0004],
        [-0.0614,  0.0772,  0.0209,  0.2040,  0.0361,  0.3044, -0.1206,  0.0621,
         -0.0655, -0.0103]], grad_fn=<AddmmBackward0>)

自定义块

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [11]:
net = MLP()
X.shape, net(X).shape

(torch.Size([2, 20]), torch.Size([2, 10]))

顺序块(自己定义)

In [24]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256),
                   nn.ReLU(),
                   nn.Linear(256, 10))

net(X)

tensor([[-0.1020,  0.1939,  0.1079,  0.1123, -0.0918,  0.1237, -0.0539,  0.1193,
          0.1067, -0.1635],
        [-0.2178,  0.2371,  0.0537, -0.0303, -0.1475,  0.0962, -0.0648,  0.0758,
          0.1237, -0.0543]], grad_fn=<AddmmBackward0>)

可以自定义类(添加新功能)

In [29]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(0.0241, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [31]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())

chimera(X)

tensor(0.1436, grad_fn=<SumBackward0>)

### 2 参数管理

##### 参数访问

In [56]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8),
                    nn.ReLU(),
                    nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.4590],
        [0.4216]], grad_fn=<AddmmBackward0>)

参数访问

In [34]:
# net[2]: 第三个模块(最后的一个线性层nn.Linear(8, 1))
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.0853,  0.1909, -0.3166, -0.3159, -0.1872, -0.3525,  0.0569,  0.1808]])), ('bias', tensor([-0.0963]))])


目标参数

In [38]:
# 访问偏置bias类型
print(type(net[2].bias))
# 访问偏置bias
print(net[2].bias)
# 访问bias的值
print(net[2].bias.data)
# 访问bias的梯度(这里没有进行计算，因此为None)
print(net[2].bias.grad)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0963], requires_grad=True)
tensor([-0.0963])
None


一次性访问所有参数

In [39]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [67]:
net.state_dict()['2.weight'].data, net.state_dict()['2.bias'].data

(tensor([[-8.4374,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000,  5.8345]]),
 tensor([0.]))

##### 自定义块的堆叠

In [53]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.1517],
        [-0.1517]], grad_fn=<AddmmBackward0>)

In [54]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


##### 内置的初始化函数

In [57]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0093,  0.0082,  0.0066, -0.0008]), tensor(0.))

In [62]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

# 对网络中所有线性层apply初始化
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

tensor([1., 1., 1., 1., 1., 1., 1., 1.])

自定义初始化

In [63]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        # 保留绝对值大于5的权重，其他设置为0
        m.weight.data *= m.weight.data.abs() >= 5
    
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-8.3040, -5.1594,  9.6554,  9.7719],
        [ 0.0000,  0.0000, -5.5495, -0.0000]], grad_fn=<SliceBackward0>)

### 3 自定义层

构造一个没有任何参数的自定义层

In [65]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

利用自定义层构建网络

In [87]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

# 输入为8维度的单精度浮点数tensor
net(torch.FloatTensor(8))


tensor([-0.1954,  0.1327,  0.0280,  0.2734,  0.2430, -0.0118,  0.2286, -0.1024,
         0.0027,  0.2531, -0.0280,  0.0396, -0.3153, -0.1427,  0.0597, -0.2364,
         0.3292,  0.0157, -0.1416, -0.3108,  0.2734, -0.3424, -0.2298,  0.1658,
         0.0005, -0.1402, -0.0146, -0.0491, -0.0230,  0.1794,  0.0673,  0.1701,
        -0.1481, -0.0802, -0.2838, -0.0805,  0.3294, -0.2966,  0.3369, -0.2659,
         0.0458, -0.1442,  0.1134, -0.2263, -0.2240, -0.2565,  0.2581,  0.0048,
         0.1901,  0.0430,  0.1892, -0.0370, -0.0931, -0.2581, -0.0204, -0.0605,
         0.2107,  0.0594, -0.1735,  0.2758, -0.1185, -0.0310,  0.3248,  0.1796,
        -0.2245,  0.3556,  0.2576,  0.3062, -0.2151,  0.1383, -0.0733,  0.2254,
        -0.1983,  0.1899,  0.2944,  0.0791,  0.2700, -0.1664, -0.2212, -0.0559,
        -0.1533, -0.0539,  0.0185, -0.1148,  0.0931,  0.2753, -0.2785,  0.1282,
         0.3109, -0.1296,  0.0905, -0.0284,  0.1937,  0.1197, -0.0610,  0.3169,
         0.3191, -0.2457, -0.2040, -0.21

### 4 读写文件

加载和保存张量

In [92]:
import torch
from torch import nn
from torch.nn import functional as FractionalMaxPool2d

x = torch.arange(4)
torch.save(x, './files from NN_basis/x-file')
x2 = torch.load('./files from NN_basis/x-file')
x2

tensor([0, 1, 2, 3])

In [122]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

net = MLP()
X = torch.randn(size=(2, 20), requires_grad=True)
Y = net(X)

(torch.Size([2, 10]), torch.Size([2, 20]))

加载和保存模型参数

In [ ]:
def myinit(m):
    if isinstance(m, nn.Linear):
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(myinit)
net.state_dict()
torch.save(net.state_dict(), './files from NN_basis/mlp.params')

重新加载模型

In [149]:
# 需要重新声明MLP
clone = MLP()

clone.load_state_dict(torch.load('./files from NN_basis/mlp.params'))

clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

验证参数

In [159]:
for key in clone.state_dict().keys() :
    print(clone.state_dict()[key] == net.state_dict()[key])

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, Tr

### GPU

In [1]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda')

(device(type='cpu'), <torch.cuda.device at 0x1c6aa90ba10>)

查询可用gpu的数量

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
x = torch.tensor([1, 2, 3])
# 查询张量创建的位置
x.device

device(type='cpu')

存储在GPU上

In [11]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [10]:
# Y存储在CPU上
Y = torch.rand(2, 3)
Y

tensor([[0.7658, 0.0198, 0.3697],
        [0.5535, 0.4857, 0.4532]])

In [12]:
# 将Y移动到GPU中
Z = Y.cuda(0)
print(Y)
print(Z)

tensor([[0.7658, 0.0198, 0.3697],
        [0.5535, 0.4857, 0.4532]])
tensor([[0.7658, 0.0198, 0.3697],
        [0.5535, 0.4857, 0.4532]], device='cuda:0')


通常将网络存放到GPU中

In [15]:
net = nn.Sequential(nn.Linear(3, 1))
# 如果有gpu,这里的try_gpu()返回torch.device('gpu')
net = net.to(device=try_gpu())
# 确定模型参数在gpu上
net[0].weight.data.device

device(type='cuda', index=0)